In [4]:
import math
import numpy as np
import torch.nn as nn
import torch

In [5]:
#this converts any positive integer to its binary form as a list
def create_binary_list_from_int(number: int):
    if number < 0 or type(number) is not int:
        raise ValueError("Only Positive integers are allowed")

    return [int(x) for x in list(bin(number))[2:]]

In [6]:
def create_int_from_binary_list():
    l = l.tolist()
    return int("".join(str(i) for i in List),2)

In [7]:
def generate_even_data(max_int: int, batch_size: int=16):
    # Get the number of binary places needed to represent the maximum number
    max_length = int(math.log(max_int, 2))

    # Sample batch_size number of integers in range 0-max_int
    sampled_integers = np.random.randint(0, int(max_int / 2), batch_size)

    # create a list of labels all ones because all numbers are even
    labels = [1] * batch_size

    # Generate a list of binary numbers for training.
    data = [create_binary_list_from_int(int(x * 2)) for x in sampled_integers]
    data = [([0] * (max_length - len(x))) + x for x in data]

    return labels, data

Generator: we would need more layers if this were an image generator; would train using random noise generated from a normal distribution and gradually upsample and reshape it until it’s the same size as the data we are trying to copy.

In [8]:
class Generator(nn.Module):

    def __init__(self, input_length: int):
        super(Generator, self).__init__()
        self.dense_layer = nn.Linear(int(input_length), int(input_length)) #single neuron model
        self.activation = nn.Sigmoid() #logistic activation function

    def forward(self, x):
        return self.activation(self.dense_layer(x))

Discriminator:

In [9]:
class Discriminator(nn.Module):
    def __init__(self, input_length: int):
        super(Discriminator, self).__init__()
        self.dense = nn.Linear(int(input_length), 1);
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense(x))

- Pass two batches of data to the model at every training step. 
    - one is random noise for the generator to generate input.
    - one is true data.

In [34]:
def train(max_int: int = 128, batch_size: int = 16, training_steps: int = 500):
    input_length = int(math.log(max_int, 2))

    # Models
    generator = Generator(input_length)
    discriminator = Discriminator(input_length)

    # Optimizers
    generator_optimizer = torch.optim.Adam(generator.parameters(), lr=0.001)
    discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.001)

    # loss
    loss = nn.BCELoss()

    for i in range(training_steps):
        # zero the gradients on each iteration
        generator_optimizer.zero_grad()

        # GENERATE RANDOM NOISE
        # Create noisy input for generator
        # Need float type instead of int
        noise = torch.randint(0, 2, size=(batch_size, input_length)).float()
        # GENERATE FAKE DATA
        generated_data = generator(noise)

        # Generate examples of even real data
        true_labels, true_data = generate_even_data(max_int, batch_size=batch_size)
        true_labels = torch.tensor(true_labels).float()
        true_data = torch.tensor(true_data).float()

        # Train the generator
        # We invert the labels here and don't train the discriminator because we want the generator
        # to make things the discriminator classifies as true.
        # PREDICTIONS OF THE DISCRIMINATOR ON THE FAKE DATA
        generator_discriminator_out = discriminator(generated_data)
        generator_loss = loss(generator_discriminator_out, true_labels) #USE TRUE LABELS 
        # BACKPROPOGATE ERROR THROUGH JUST THE GENERATOR
        generator_loss.backward()
        generator_optimizer.step()

        # Train the discriminator on the true/generated data
        # 
        discriminator_optimizer.zero_grad()
        true_discriminator_out = discriminator(true_data)
        true_discriminator_loss = loss(true_discriminator_out, true_labels)

        # add .detach() here think about this
        generator_discriminator_out = discriminator(generated_data.detach())
        generator_discriminator_loss = loss(generator_discriminator_out, torch.zeros(batch_size))
        discriminator_loss = (true_discriminator_loss + generator_discriminator_loss) / 2
        discriminator_loss.backward()
        discriminator_optimizer.step()
        if (i % 50 == 0):
            #print("Step "+str(i)+": ")
            print(list(map(int, torch.round(generated_data).tolist())))
            #print(int("".join(str(i) for i in torch.round(generated_data).tolist()),2))
    return generator

In [35]:
gen = train()

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [43]:
gen

Generator(
  (dense_layer): Linear(in_features=7, out_features=7, bias=True)
  (activation): Sigmoid()
)